In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from keras.models import load_model

In [2]:
# Loads saved model
model = load_model('multiple_input_multiple_datasets_model.h5')

In [3]:
def get_model_prediction(dataset_filename_to_read):
  data = pd.read_csv(dataset_filename_to_read)
  data = data.iloc[::-1]
  data = data.drop('date', axis=1)
  data = data.values

  normalizing_scaler = preprocessing.MinMaxScaler()
  normalized_data = normalizing_scaler.fit_transform(data)

  # Get y_normalizer
  # Get unscaled stock open price from original file data
  next_day_open_values = np.array([data[:,0][i + 30].copy() for i in range(len(data) - 30)])
  next_day_open_values = np.expand_dims(next_day_open_values, -1)
  y_normaliser = preprocessing.MinMaxScaler()
  y_normaliser.fit(next_day_open_values)

  # Get most recent 30 stock market open prices to feed to model, as well as moving average of values
  ohlcv_one = normalized_data[-30:]
  mov_avg_one = np.mean(ohlcv_one[:,3])
  # Prepare values to be fed into model
  ohlcv_one = np.array([ohlcv_one])
  mov_avg_one = np.array([mov_avg_one])

  predicted_price_tomorrow = np.squeeze(y_normaliser.inverse_transform(model.predict([ohlcv_one, mov_avg_one])))
  return predicted_price_tomorrow

In [8]:
user_query = "FB"
fileName = user_query + "_daily.csv"
next_day_opening_stock_price = get_model_prediction(fileName)

array(253.63635, dtype=float32)

In [12]:
print(type(next_day_opening_stock_price))
print(next_day_opening_stock_price)
next_day_opening_stock_price = int(next_day_opening_stock_price)
print(type(next_day_opening_stock_price))
print(next_day_opening_stock_price)

<class 'numpy.ndarray'>
253.63635
<class 'int'>
253


In [ ]:
# csv file path of dataset testing model on. Top of dataset is most recent stock prices
dataset_filename_to_read = 'NFLX_daily.csv'
data = pd.read_csv(dataset_filename_to_read)
data = data.iloc[::-1]
data = data.drop('date', axis=1)
print(data.head())
print(data.shape)
print(type(data))

In [ ]:
data = data.values
print(type(data))

In [ ]:
most_recent_open_values = data[-10:,0]
print(most_recent_open_values)

In [ ]:
plt.plot(data[-60:,0])

In [ ]:
normalizing_scaler = preprocessing.MinMaxScaler()
normalized_data = normalizing_scaler.fit_transform(data)
print("Normalized data")
print(normalized_data[-5:-1,:])
print(normalized_data.shape)

In [ ]:
# Get y_normalizer
# Get unscaled stock open price from original file data
next_day_open_values = np.array([data[:,0][i + 30].copy() for i in range(len(data) - 30)])
next_day_open_values = np.expand_dims(next_day_open_values, -1)
print("Next day open values unscaled:", next_day_open_values.shape)

y_normaliser = preprocessing.MinMaxScaler()
y_normaliser.fit(next_day_open_values)

In [ ]:
# ohlcv_one is first 50 readings (days) of open/high/low/close/volume values
ohlcv_one = normalized_data[-30:]
#ohlcv_one = np.array([ohlcv_one])
print(ohlcv_one.shape)
print(ohlcv_one[0][0])

ohlcv_two = normalized_data[-60:-30]
#ohlcv_two = np.array([ohlcv_two])

In [ ]:
mov_avg_one = np.mean(ohlcv_one[:,3]) # closing price average, so average closing price from first 50 days
print(type(mov_avg_one))
print(mov_avg_one)

mov_avg_two = np.mean(ohlcv_two[:,3])

In [ ]:
def test_model(ohlcvItem, movAvgItem):
    normalized_price_today = ohlcvItem[0][0]
    normalized_price_today = np.array([[normalized_price_today]])
    price_today = y_normaliser.inverse_transform(normalized_price_today)
    
    ohlcvItem = np.array([ohlcvItem])
    movAvgItem = np.array([movAvgItem])
    print(ohlcvItem.shape)
    print(movAvgItem.shape)

    predicted_price_tomorrow = np.squeeze(y_normaliser.inverse_transform(model.predict([ohlcvItem, movAvgItem])))
    print("Actual price: ", price_today)
    print("Predicted price: ", predicted_price_tomorrow)

In [ ]:
test_model(ohlcv_one, mov_avg_one)

In [ ]:
test_model(ohlcv_two, mov_avg_two)

## Getting pictures for presentation

In [ ]:
history_days = 30

In [ ]:
def csv_to_dataset(csv_file_path):
  file_data = pd.read_csv(csv_file_path)
  file_data = file_data.iloc[::-1] # Reverse order, most recent values last, want to be predicting into future, not past
  file_data = file_data.drop('date', axis=1)
  file_data = file_data.drop(0, axis=0)
  print("File data DataFrame:", file_data.shape)
  print(file_data.head())
  file_data = file_data.values
  
  normalizing_scaler = preprocessing.MinMaxScaler()
  normalized_data = normalizing_scaler.fit_transform(file_data)
  print()
  print("Normalized data")
  print(normalized_data[0:5,:])
  
  # Data is in order of: Open stock value, high value, low, close, and volume - ohlcv
  # Creates array of 5x50-value array windows, each one will be a training input into model
  ohlcv_histories_normalised = np.array([normalized_data[i : i + history_days].copy() for i in range(len(normalized_data) - history_days)])
  print()
  print("Normalized inputs", ohlcv_histories_normalised.shape)
  #print(ohlcv_histories_normalised[0:2,0:5])
  
  # Get scaled stock open price values, which model is predicting
  next_day_open_values_normalised = np.array([normalized_data[:,0][i + history_days].copy() for i in range(len(normalized_data) - history_days)])
  next_day_open_values_normalised = np.expand_dims(next_day_open_values_normalised, -1)
  #print()
  print("Next day open values scaled:", next_day_open_values_normalised.shape)
  
  # Get unscaled stock open price from original file data
  next_day_open_values = np.array([file_data[:,0][i + history_days].copy() for i in range(len(file_data) - history_days)])
  next_day_open_values = np.expand_dims(next_day_open_values, -1)
  print("Next day open values unscaled:", next_day_open_values.shape)

  y_normaliser = preprocessing.MinMaxScaler()
  y_normaliser.fit(next_day_open_values)

  # Moving average technical indicator of stock price input
  moving_averages = []
  for his in ohlcv_histories_normalised:
    sma = np.mean(his[:,3]) # Using closing price of the stocks for the moving average, not open price
    moving_averages.append(np.array([sma]))

  moving_averages = np.array(moving_averages) # Convert to numpy array
  moving_averages_scaler = preprocessing.MinMaxScaler() # Scale with min-max scaler
  moving_averages_normalised = moving_averages_scaler.fit_transform(moving_averages)

  assert ohlcv_histories_normalised.shape[0] == next_day_open_values_normalised.shape[0] == moving_averages_normalised.shape[0]
  return ohlcv_histories_normalised, moving_averages_normalised, next_day_open_values_normalised, next_day_open_values, y_normaliser

In [ ]:
def multiple_csv_to_dataset(test_set_name):
  import os
  ohlcv_histories = 0
  moving_averages = 0
  next_day_open_values = 0
  # For each company stock dataset in directory, add data to training dataset
  for csv_file_path in list(filter(lambda x: x.endswith('daily.csv'), os.listdir('./'))):
    if not csv_file_path == test_set_name:
      print(csv_file_path)
      if type(ohlcv_histories) == int:
        ohlcv_histories, moving_averages, next_day_open_values, _, _ = csv_to_dataset(csv_file_path)
      else:
        a, b, c, _, _ = csv_to_dataset(csv_file_path)
        ohlcv_histories = np.concatenate((ohlcv_histories, a), 0)
        moving_averages = np.concatenate((moving_averages, b), 0)
        next_day_open_values = np.concatenate((next_day_open_values, c), 0)

  ohlcv_train = ohlcv_histories
  mov_avg_train = moving_averages
  open_prices_train = next_day_open_values

  ohlcv_test, mov_avg_test, open_prices_test, unscaled_open_prices_test, y_normaliser = csv_to_dataset(test_set_name)

  return ohlcv_train, mov_avg_train, open_prices_train, ohlcv_test, mov_avg_test, open_prices_test, unscaled_open_prices_test, y_normaliser

In [ ]:
#ohlcv_train, mov_avg_train, open_prices_train, ohlcv_test, mov_avg_test, open_prices_test, unscaled_open_prices_test, y_normaliser = multiple_csv_to_dataset('FB_daily.csv')

In [ ]:
#open_prices_test_predicted = model.predict([ohlcv_test, mov_avg_test])
#open_prices_test_predicted = y_normaliser.inverse_transform(open_prices_test_predicted)

In [ ]:
#import matplotlib.pyplot as plt

In [ ]:
#plt.gcf().set_size_inches(22, 15, forward=True)
#start = 0
#end = -1
#real = plt.plot(unscaled_open_prices_test[start:end], label='real')
#pred = plt.plot(open_prices_test_predicted[start:end], label='predicted')

#plt.legend(['Real', 'Predicted'])
#plt.show()